## Data Cleaning and Merging

This section walks through the data ingestion and transformation process to prepare the dataset for analysis. Key steps include:

- Load multiple yearly datasets from CSVs
- Clean the job postings dataset
- Integrate exchange rate data to convert salaries to EUR
- Merge country metadata for regional classification
- Export the cleaned dataset for further exploration

### Import Libraries

In [94]:
from pathlib import Path
import ast

import requests
import pandas as pd
from datetime import datetime
from tqdm import tqdm

### Load multiple yearly datasets from CSVs

In [95]:
raw_data_dir = Path.cwd().parents[1] / 'Raw_Data'
csv_files = list(raw_data_dir.glob('*data_jobs*.csv'))

dfs = [pd.read_csv(f) for f in csv_files]
df = pd.concat(dfs, ignore_index=True)

df.head(3)

,job_title_short,job_title,job_location,job_via,job_schedule_type,job_work_from_home,search_location,job_posted_date,job_no_degree_mention,job_health_insurance,job_country,salary_rate,salary_year_avg,salary_hour_avg,company_name,job_skills,job_type_skills
0,Data Analyst,"Summer Internship -Data Analyst Intern, Risk M...","Marlborough, MA",via Boatingrevealed.com,"Full-time, Part-time, and Internship",False,"New York, United States",2024-01-01 00:00:01,False,True,United States,NaN,NaN,NaN,BJ's Wholesale Club,['excel'],{'analyst_tools': ['excel']}
1,Data Analyst,"Staff Data Analyst Operations, Infrastructure ...","Fremont, CA",via ClimateTechList,Full-time,False,"California, United States",2024-01-01 00:00:11,True,False,United States,NaN,NaN,NaN,Tesla,"['tableau', 'flow']","{'analyst_tools': ['tableau'], 'other': ['flow']}"
2,Data Analyst,Junior Data Analyst - Entry Level,"Waco, TX",via ZipRecruiter,Full-time and Part-time,False,"Texas, United States",2024-01-01 00:00:15,True,False,United States,NaN,NaN,NaN,Next Recruiting,NaN,NaN


### Clean the job postings dataset

Removing duplicates, rows with empty salary and skills (as they are a key value in this analysis), converting data types and filtering only needed job titjes.

In [96]:
# Basic cleaning
df = df.drop_duplicates()
df = df.dropna(subset=['salary_year_avg', 'job_skills'])

# Type conversions
df['job_posted_date'] = pd.to_datetime(df['job_posted_date'], errors='coerce')
df['job_skills'] = df['job_skills'].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else x)

# Filtering
data_jobs = {'Data Analyst', 'Senior Data Analyst', 'Data Scientist'}
df = df[df['job_title_short'].isin(data_jobs)].copy()

#### Remove salary outliers

Outliers are removed per job title & country to avoid distorted salary distributions. If a country has fewer than 10 postings for a given job title, outliers are removed using global IQR instead.

In [97]:
# Calculate global IQR
Q1_global = df['salary_year_avg'].quantile(0.25)
Q3_global = df['salary_year_avg'].quantile(0.75)
IQR_global = Q3_global - Q1_global
lower_bound_global = Q1_global - 1.5 * IQR_global
upper_bound_global = Q3_global + 1.5 * IQR_global

filtered_groups = []

# Iterate over groups manually
for (country, title), group in df.groupby(['job_country', 'job_title_short']):
    if len(group) >= 10:
        Q1 = group['salary_year_avg'].quantile(0.25)
        Q3 = group['salary_year_avg'].quantile(0.75)
        IQR = Q3 - Q1
        lower = Q1 - 1.5 * IQR
        upper = Q3 + 1.5 * IQR
    else:
        lower = lower_bound_global
        upper = upper_bound_global

    filtered = group[
        (group['salary_year_avg'] >= lower) &
        (group['salary_year_avg'] <= upper)
    ]
    filtered_groups.append(filtered)

# Combine all groups back
df_filtered = pd.concat(filtered_groups, ignore_index=True)

print(f'Original dataset size: {len(df)}')
print(f'Filtered dataset size: {len(df_filtered)}')

Original dataset size: 22089
Filtered dataset size: 21492


### Integrate exchange rate data to convert salaries to EUR

TTo convert salaries to a more relevant local currency (EUR), collect USD→EUR exchange rates for each year starting from 2023 using a public API. These rates will later be used to calculate both annual and monthly salaries in EUR.

In [98]:
current_year = datetime.now().year

# Function to get exchange rate from Frankfurter API
def get_usd_to_eur_rate_frankfurter(date_str):
    url = f"https://api.frankfurter.app/{date_str}?from=USD&to=EUR"
    response = requests.get(url)
    data = response.json()
    if 'rates' in data and 'EUR' in data['rates']:
        return data['rates']['EUR']
    return None

# Filter unique dates: only from 2023 to current year
unique_dates = sorted(
    date for date in df_filtered['job_posted_date'].dt.date.unique()
    if date.year >= 2023 and date.year <= current_year
)

# Fetch exchange rates only for filtered dates
ex_rates = []
for date_obj in tqdm(unique_dates, desc='Fetching exchange rates'):
    date_str = date_obj.strftime('%Y-%m-%d')
    rate = get_usd_to_eur_rate_frankfurter(date_str)
    ex_rates.append({'job_posted_date': date_obj, 'usd_to_eur': rate})

# Save results
df_ex_rate = pd.DataFrame(ex_rates).round(4)
df_ex_rate.to_csv(raw_data_dir / 'ex_rate_daily.csv', index=False)

df_ex_rate.head(3)

Fetching exchange rates: 100%|██████████| 728/728 [02:57<00:00,  4.09it/s]


,job_posted_date,usd_to_eur
0,2023-01-01,0.9376
1,2023-01-02,0.9361
2,2023-01-03,0.9483


In [99]:
# Merge Exchange Rate. Convert job_year_avg in EUR
df_Final = df_filtered.rename(columns={'salary_year_avg': 'salary_year_avg_usd'})

df_Final['job_posted_date'] = pd.to_datetime(df_Final['job_posted_date'])
df_ex_rate['job_posted_date'] = pd.to_datetime(df_ex_rate['job_posted_date'])

df_Final['job_posted_date'] = df_Final['job_posted_date'].dt.date
df_ex_rate['job_posted_date'] = df_ex_rate['job_posted_date'].dt.date

df_Final = df_Final.merge(df_ex_rate, on='job_posted_date', how='left')

df_Final['salary_year_avg_eur'] = (df_Final['salary_year_avg_usd'] * df_Final['usd_to_eur']).round(2)
df_Final['salary_month_avg_eur'] = (df_Final['salary_year_avg_eur'] / 12).round(2)

df_Final.loc[1:3, ['job_title_short', 'job_posted_date', 'usd_to_eur', 'salary_year_avg_usd', 'salary_year_avg_eur', 'salary_month_avg_eur']]

,job_title_short,job_posted_date,usd_to_eur,salary_year_avg_usd,salary_year_avg_eur,salary_month_avg_eur
1,Data Analyst,2024-12-14,0.9508,192000.0,182553.60,15212.80
2,Data Analyst,2023-07-05,0.9192,56700.0,52118.64,4343.22
3,Data Analyst,2023-11-17,0.9198,43200.0,39735.36,3311.28


### Merge country metadata for regional classification

In [100]:
df_EU = pd.read_csv(raw_data_dir / 'EU_Countries_dict.csv', delimiter=';')

df_Final = df_Final.merge(df_EU, how='left', left_on='job_country', right_on='country')
pd.set_option('future.no_silent_downcasting', True)
df_Final['is_eu'] = df_Final['is_eu'].fillna(False).astype(bool)

df_Final.loc[1001:1003, ['job_title_short', 'job_country', 'is_eu']]

,job_title_short,job_country,is_eu
1001,Data Scientist,India,False
1002,Data Scientist,India,False
1003,Data Scientist,India,False


In [101]:
# Add Region grouping column
def group_country(row):
    if row['job_country'] == 'United States':
        return 'US'
    elif row['is_eu'] == True:
        return 'EU'
    else:
        return 'Other'
    
df_Final['region_group'] = df_Final.apply(group_country, axis=1)

df_Final.loc[1001:1003, ['region_group', 'job_country', 'salary_month_avg_eur']]

,region_group,job_country,salary_month_avg_eur
1001,Other,India,12022.40
1002,Other,India,4993.92
1003,Other,India,2890.00


### Export the cleaned dataset for further exploration

After cleaning and merging, the final dataset is exported to CSV for downstream analysis. Temporary or unused columns are dropped to reduce noise.

In [102]:
df_Final.drop(columns=['usd_to_eur', 'ISO', 'salary_year_avg_usd', 'salary_hour_avg'], inplace=True)
df_Final.to_pickle(raw_data_dir / 'df_Final_2.pkl')

df_Final.head(3)

,job_title_short,job_title,job_location,job_via,job_schedule_type,job_work_from_home,search_location,job_posted_date,job_no_degree_mention,job_health_insurance,job_country,salary_rate,company_name,job_skills,job_type_skills,salary_year_avg_eur,salary_month_avg_eur,country,is_eu,region_group
0,Senior Data Analyst,"Senior Data Analyst, Product Analytics",Anywhere,via Jobgether,Full-time,True,Afghanistan,2024-04-12,False,False,Afghanistan,year,Remote,"[sql, python, aws, snowflake, looker]","{'analyst_tools': ['looker'], 'cloud': ['aws',...",54685.10,4557.09,NaN,False,Other
1,Data Analyst,Data Analyst (HR & Finance),Anywhere,Jobgether,Full-time,True,Albania,2024-12-14,True,False,Albania,year,Smartcat,"[c, excel, sheets]","{'analyst_tools': ['excel', 'sheets'], 'progra...",182553.60,15212.80,NaN,False,Other
2,Data Analyst,Junior Data Quality Specialist with financial ...,"Tirana, Albania",via Ai-Jobs.net,Full-time,False,Albania,2023-07-05,False,False,Albania,year,AUTO1 Group,"[sql, oracle, excel]","{'analyst_tools': ['excel'], 'cloud': ['oracle...",52118.64,4343.22,NaN,False,Other
